In [28]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision.transforms as transforms
from ASLDataset import ASLDataset
import torch.optim as optim
import time
from torchsummary import summary
import torchvision.models as models
from torchvision.io import read_image
from matplotlib.image import imread

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


resnet = models.resnet18(pretrained=False)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 29)
resnet.load_state_dict(torch.load(
    './trained_models/CustomResNet18/resnet_raw.pth'))
resnet.eval()
resnet.to(device)
print(device)

summary(resnet,(3, 200, 200))


cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           9,408
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((224, 224)),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
)

image = imread('./D.jpg')
image = transform(image)
image = image.to(device).unsqueeze(0)

print(image.shape)


ValueError: If size is a sequence, it should have 1 or 2 values

In [8]:
output = resnet(image)
_, predicted = torch.max(output.data, 1)
softmax = nn.Softmax(dim=1)
print(output)
print(F.softmax(output, dim=1))
print(predicted.item())

tensor([[-0.4001, -0.0887,  0.0046,  2.6850, -0.1702,  1.3317,  0.2594, -0.2203,
          0.3193, -0.4518,  0.3221,  1.0835, -0.6366,  0.0960,  0.4825,  0.8003,
         -0.0745, -0.0293, -0.3528, -0.4721,  0.2172,  0.2974, -0.3147, -0.5444,
         -0.2049, -0.2990, -0.2992, -0.3494,  0.2656]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([[0.0142, 0.0194, 0.0213, 0.3103, 0.0179, 0.0802, 0.0274, 0.0170, 0.0291,
         0.0135, 0.0292, 0.0626, 0.0112, 0.0233, 0.0343, 0.0471, 0.0197, 0.0206,
         0.0149, 0.0132, 0.0263, 0.0285, 0.0155, 0.0123, 0.0172, 0.0157, 0.0157,
         0.0149, 0.0276]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
3
